In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt
!pip -q install -r requirements.txt

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-03 16:11:32--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  21.0MB/s    in 95s     

2021-01-03 16:13:06 (19.5 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-03 16:13:06--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: 

In [3]:
!pip -q install -r requirements.txt

In [4]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

In [5]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

body_parts = ['face_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
          
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:04<00:00, 1549.96it/s]


In [6]:
src['sign'].shape

torch.Size([43, 252])

In [7]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 252]
list_doc['training']['recognition_loss_weight'] = 20

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
!python -m signjoey train configs/sign.yaml

Streaming output truncated to the last 5000 lines.
2021-01-03 16:35:25,380 	Text Alignment  :	I  I         I    S    S    S                D         S      S      S      S      S     S             I    I                     S               
2021-01-03 16:35:25,380 ========================================================================================================================
2021-01-03 16:35:34,624 [Epoch: 014 Step: 00003000] Batch Recognition Loss:  12.706508 => Gls Tokens per Sec:     2786 || Batch Translation Loss:   9.250466 => Txt Tokens per Sec:     5695 || Lr: 0.001000
2021-01-03 16:35:59,248 Hooray! New best validation result [eval_metric]!
2021-01-03 16:35:59,249 Saving new checkpoint.
2021-01-03 16:35:59,683 Validation result at epoch  14, step     3000: duration: 25.0584s
	Recognition Beam Size: 1	Translation Beam Size: 1	Translation Beam Alpha: -1
	Recognition Loss: 609.68506	Translation Loss: 19746.70117	PPL: 12.34119
	Eval Metric: BLEU
	WER 38.56	(DEL: 29.41,	INS: